# Object Detection in Video

Prerequisites: 
1. Train Custom Vision Model
2. Save frames of video 

Each frame is passed to the Custom Vision Prediction API to generate the prediction tag, prediction probability and the bounding boxes of each object. These details are applied to each frame using the Python library cv2, and the resulting image is saved to another local folder.

## 1. Import Libraries & Keys

In [ ]:
import azure
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
import cv2
import numpy as np
import math
from decimal import *
import os, sys
from dotenv import load_dotenv

In [ ]:
load_dotenv(verbose=True)

In [ ]:
# Resource ID of Custom Vision Azure Resource
prediction_resource_id = os.getenv('prediction_resource_id')

# Custom Vision Endpoint
ENDPOINT = os.getenv('ENDPOINT')

# Keys from your Custom Vision project where you trained the model
training_key = os.getenv('training_key')
prediction_key = os.getenv('prediction_key')

# ID and Iteration Name of the Custom Vision project where you trained the model
# and the iteration name of the published model
project_id = os.getenv('project_id')
publish_iteration_name = os.getenv('publish_iteration_name')

## 2. Run Prediction over Sample Files

In [ ]:
# The local path to the sample files
path = os.getenv('load_path')
dirs = os.listdir(path)

# Initialise the count
count = 0

# Initialise the predictor
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)

In [ ]:
# For each file in the sample file folder...
for f in dirs:

    # read the image in and run the prediction
    img = cv2.imread(path+f, cv2.IMREAD_UNCHANGED)
    img = cv2.pyrDown(img)
    im_h, im_w, im_c = img.shape

    with open(path+f, mode="rb") as test_data:
        results = predictor.detect_image(project_id, publish_iteration_name, test_data)

    # Display the results.    
    for prediction in results.predictions:
        print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
    
        # only put bounding box if probability is more than 51%
        d = Decimal(prediction.probability)
        if d >= 0.51:
            pred_prob = str(round(d * 100))   
            print("pred_prob: {}".format(pred_prob))

            x = int(math.floor(prediction.bounding_box.left * im_w))
            y = int(math.floor(prediction.bounding_box.top * im_h))
            w = int(math.floor(prediction.bounding_box.width * im_w))
            h = int(math.floor(prediction.bounding_box.height * im_h))

            # red formatting for Apache
            if prediction.tag_name == "Apache AH-64":
                # rectangle
                img_box = cv2.rectangle(img,(x,y),(x+w,y+h),(204,0,0),2) 
                # apply label
                text_label = '{}%'.format(pred_prob)
                cv2.putText(img,prediction.tag_name,(x,y+h+20),0,0.4,(204,0,0))
                cv2.putText(img,text_label,(x,y+h+35),0,0.4,(204,0,0))
            
            # Green formatting for Chinook  
            elif prediction.tag_name == "CH-47 Chinook":
                # rectangle
                img_box = cv2.rectangle(img,(x,y),(x+w,y+h),(0,102,0),2) 
                # apply label
                text_label = '{}%'.format(pred_prob)

                cv2.putText(img,prediction.tag_name,(x,y+h+20),0,0.4,(0,102,0))
                cv2.putText(img,text_label,(x,y+h+35),0,0.4,(0,102,0))

            # Blue formatting for Agusta Bell
            elif prediction.tag_name == "Agusta Bell UH-1":
                # rectangle
                img_box = cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2) 
                # apply label
                text_label = '{}%'.format(pred_prob)
                #cv2.putText(img,text_label,(x,y+h+20),0,0.4,(255,0,0))
                cv2.putText(img,prediction.tag_name,(x,y+h+20),0,0.4,(0,0,255))
                cv2.putText(img,text_label,(x,y+h+35),0,0.4,(0,0,255))

            # White formatting for everything else
            else:
                # rectangle
                img_box = cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),2) 
                # apply label
                text_label = '{}%'.format(pred_prob)
                #cv2.putText(img,text_label,(x,y+h+20),0,0.4,(255,0,0))
                cv2.putText(img,prediction.tag_name,(x,y+h+20),0,0.4,(255,255,255))
                cv2.putText(img,text_label,(x,y+h+35),0,0.4,(255,255,255))

    # save the file to a local folder with the formatted bounding boxes and text, then move to the next image. 
    image_to_write = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    file_to_be_saved = os.getenv('file_to_be_saved')
    save_file = cv2.imwrite(file_to_be_saved.format(count),image_to_write)
    count += 1
    
print("Finished!")